## Car sales

We have an app to attract new customers. In that app, customer can quickly find out the market value of his car. We need to build the model to determine the value. 

We are interested in:

* the quality of the prediction
* the speed of the prediction
* the time required for training

In [1]:
import pandas as pd
import time
from sklearn.metrics import  make_scorer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('/datasets/car_data.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Mileage              354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


We have dataset with 354369 entries. There are 16 columns : 7 columns are integer, 9 - object type. 

In [3]:
df[df.duplicated(keep=False)].shape

(524, 16)

There are 524 duplicates. We'll drop duplicates except for the first occurrence using drop_duplicates() method.

In [4]:
df = df.drop_duplicates()

In [5]:
df.isna().mean()

DateCrawled          0.000000
Price                0.000000
VehicleType          0.105855
RegistrationYear     0.000000
Gearbox              0.056000
Power                0.000000
Model                0.055636
Mileage              0.000000
RegistrationMonth    0.000000
FuelType             0.092879
Brand                0.000000
NotRepaired          0.200914
DateCreated          0.000000
NumberOfPictures     0.000000
PostalCode           0.000000
LastSeen             0.000000
dtype: float64

We have 5 columns with missing values: VehicleType, Gearbox, Model, FuelType, NotRepaired. The information in these columns came from customers. We can't fill these missing values without client.

Let's drop all missing values.

In [6]:
df = df.dropna()
df.shape

(245567, 16)

After droping missing values we stayed with about 70% of the data. 

In oder to find market value of the car we don't need information about clients' account, we need only car characteristics. So let's drop these columns: DateCrawled, DateCreated, NumberOfPictures, PostalCode, LastSeen.  

In [7]:
df = df.drop(['DateCrawled','DateCreated','NumberOfPictures','PostalCode','LastSeen'], axis=1)
df.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no
5,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes
6,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no
7,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no


Our target variable "price" is numerical, so we have regression task. 

First, transform categorical features into numerical features using One-Hot Encoding (OHE). We'll call pd.get_dummies() for all data and set drop_first = True to avoid the dummy trap.

In [8]:
df_ohe = pd.get_dummies(df, drop_first=True)

In [9]:
target = df_ohe['Price']
features = df_ohe.drop('Price', axis=1)

# split the data into a training set and validation set at a ratio of 75:25
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
    
# scale the numeric data using StandardScaler() from sklearn.preprocessing module.
numeric = ['RegistrationYear','Power','Mileage','RegistrationMonth']

scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])

We'll use RMSE to evaluate different models.

In [10]:
def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)**0.5

Let's compare four models: Linear Regression, Random Forest, LightGBM, CatBoost.  

* #### Linear Regression

In [11]:
regressor = LinearRegression()  
regressor.fit(features_train, target_train)

predicted_valid = regressor.predict(features_valid)

rmse_linear_reg = rmse(target_valid, predicted_valid)
print('RMSE:', rmse_linear_reg)


RMSE: 2713.527208953366


* #### Random Forest

Let's tune hyperparameters for Random Forest using RandomizedSearch.

In [12]:
model = RandomForestRegressor()

parameters = {'max_depth'     : [6,10],
              'n_estimators'  : [40, 50] }
    
randm = RandomizedSearchCV(estimator=model, param_distributions = parameters,
                           scoring='neg_mean_squared_error', cv = 3, n_iter = 10, 
                           n_jobs=-1, verbose = 2, random_state=12345)

randm_results = randm.fit(features_train, target_train)


print("\n The best score:\n ", randm_results.best_score_)

print("\n The best parameters:\n ", randm_results.best_params_)

# Predict after fitting RandomizedSearchCV with best parameters
y_pred = randm .predict(features_valid)
 
score_random_forest = rmse(target_valid, y_pred)

print("\n Score on test data:\n ", score_random_forest)


Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] n_estimators=40, max_depth=6 ....................................


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..................... n_estimators=40, max_depth=6, total=  41.4s
[CV] n_estimators=40, max_depth=6 ....................................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   41.4s remaining:    0.0s


[CV] ..................... n_estimators=40, max_depth=6, total=  42.9s
[CV] n_estimators=40, max_depth=6 ....................................
[CV] ..................... n_estimators=40, max_depth=6, total=  42.2s
[CV] n_estimators=50, max_depth=6 ....................................
[CV] ..................... n_estimators=50, max_depth=6, total=  53.3s
[CV] n_estimators=50, max_depth=6 ....................................
[CV] ..................... n_estimators=50, max_depth=6, total=  53.1s
[CV] n_estimators=50, max_depth=6 ....................................
[CV] ..................... n_estimators=50, max_depth=6, total=  52.7s
[CV] n_estimators=40, max_depth=10 ...................................
[CV] .................... n_estimators=40, max_depth=10, total= 1.0min
[CV] n_estimators=40, max_depth=10 ...................................
[CV] .................... n_estimators=40, max_depth=10, total= 1.0min
[CV] n_estimators=40, max_depth=10 ...................................
[CV] .

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 11.8min finished



 The best score:
  -3788300.102208344

 The best parameters:
  {'n_estimators': 40, 'max_depth': 10}

 Score on test data:
  1946.4253217634284


For LightGBM and CatBoost we'll convert all 'object' data types to 'categorical' data types

In [13]:
target = df['Price']
features = df.drop('Price', axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(features, target, test_size=0.25, random_state=12345)

In [14]:
cat_features = ['VehicleType','RegistrationYear','Gearbox','Model', 'RegistrationMonth','FuelType','Brand','NotRepaired']

for header in cat_features:
    X_train[header] = X_train[header].astype('category')
    X_valid[header] = X_valid[header].astype('category')

* #### CatBoost

For tuning hyperparameters for CatBoost and LightGBM we'll use RandomizedSearch again.

In [15]:
model = CatBoostRegressor()

parameters = {'depth'         : [6,10],
              'learning_rate' : [0.03, 0.1],
              'n_estimators'  : [60,100] }
    
randm = RandomizedSearchCV(estimator=model, param_distributions = parameters, scoring='neg_mean_squared_error',  
                               cv = 3, n_iter = 10, n_jobs=-1, verbose = 2, random_state=12345)

randm_results=randm.fit(X_train, y_train,cat_features=cat_features)


print("\n The best score:\n ", randm_results.best_score_)

print("\n The best parameters:\n ", randm_results.best_params_)

# Predict after fitting RandomizedSearchCV with best parameters
y_pred = randm .predict(X_valid)
 
score_catboost = rmse(y_valid, y_pred)

print("\n Score on test data:\n ",score_catboost)


Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] n_estimators=60, learning_rate=0.03, depth=6 ....................


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.


0:	learn: 4624.2096442	total: 192ms	remaining: 11.3s
1:	learn: 4533.8704676	total: 392ms	remaining: 11.4s
2:	learn: 4445.8074521	total: 684ms	remaining: 13s
3:	learn: 4362.0110640	total: 883ms	remaining: 12.4s
4:	learn: 4279.5737430	total: 1.09s	remaining: 12s
5:	learn: 4200.2157723	total: 1.29s	remaining: 11.6s
6:	learn: 4123.4443283	total: 1.49s	remaining: 11.3s
7:	learn: 4049.4382654	total: 1.78s	remaining: 11.6s
8:	learn: 3978.8516632	total: 1.98s	remaining: 11.2s
9:	learn: 3909.9792343	total: 2.18s	remaining: 10.9s
10:	learn: 3845.4083412	total: 2.38s	remaining: 10.6s
11:	learn: 3783.7934624	total: 2.58s	remaining: 10.3s
12:	learn: 3722.4359959	total: 2.78s	remaining: 10.1s
13:	learn: 3664.0844418	total: 2.98s	remaining: 9.8s
14:	learn: 3607.7736287	total: 3.19s	remaining: 9.56s
15:	learn: 3554.4493895	total: 3.38s	remaining: 9.3s
16:	learn: 3501.5494646	total: 3.58s	remaining: 9.07s
17:	learn: 3451.1937425	total: 3.87s	remaining: 9.03s
18:	learn: 3403.0910081	total: 4.07s	remaini

[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   14.3s remaining:    0.0s


0:	learn: 4629.7432785	total: 148ms	remaining: 8.73s
1:	learn: 4539.6979674	total: 347ms	remaining: 10.1s
2:	learn: 4451.8403817	total: 550ms	remaining: 10.4s
3:	learn: 4367.5244479	total: 748ms	remaining: 10.5s
4:	learn: 4284.5992199	total: 1.04s	remaining: 11.5s
5:	learn: 4206.4573953	total: 1.24s	remaining: 11.2s
6:	learn: 4128.9779651	total: 1.44s	remaining: 10.9s
7:	learn: 4056.8637379	total: 1.64s	remaining: 10.7s
8:	learn: 3984.8135172	total: 1.84s	remaining: 10.4s
9:	learn: 3916.0311446	total: 2.04s	remaining: 10.2s
10:	learn: 3850.5925737	total: 2.24s	remaining: 9.99s
11:	learn: 3788.0361598	total: 2.44s	remaining: 9.78s
12:	learn: 3728.1490807	total: 2.65s	remaining: 9.59s
13:	learn: 3671.1592798	total: 2.84s	remaining: 9.34s
14:	learn: 3613.0704123	total: 3.04s	remaining: 9.13s
15:	learn: 3560.0160209	total: 3.24s	remaining: 8.91s
16:	learn: 3509.8969247	total: 3.53s	remaining: 8.93s
17:	learn: 3460.4597190	total: 3.73s	remaining: 8.7s
18:	learn: 3412.2286813	total: 3.93s	re

...

49:	learn: 1784.7287991	total: 16.2s	remaining: 3.23s
50:	learn: 1782.0609966	total: 16.5s	remaining: 2.9s
51:	learn: 1779.8128097	total: 16.8s	remaining: 2.58s
52:	learn: 1777.6923109	total: 17.1s	remaining: 2.25s
53:	learn: 1774.7803271	total: 17.5s	remaining: 1.94s
54:	learn: 1769.0815249	total: 17.8s	remaining: 1.61s
55:	learn: 1765.5985659	total: 18.1s	remaining: 1.29s
56:	learn: 1763.1513543	total: 18.4s	remaining: 966ms
57:	learn: 1760.4127512	total: 18.7s	remaining: 646ms
58:	learn: 1758.4720387	total: 19.1s	remaining: 323ms
59:	learn: 1756.8470544	total: 19.4s	remaining: 0us
[CV] ..... n_estimators=60, learning_rate=0.1, depth=10, total=  21.4s
[CV] n_estimators=60, learning_rate=0.1, depth=10 ....................
0:	learn: 4369.7177814	total: 291ms	remaining: 17.2s
1:	learn: 4073.7295014	total: 594ms	remaining: 17.2s
2:	learn: 3815.5321663	total: 894ms	remaining: 17s
3:	learn: 3590.8433743	total: 1.2s	remaining: 16.8s
4:	learn: 3390.3315154	total: 1.59s	remaining: 17.5s
5:	le

77:	learn: 1714.1540243	total: 25.5s	remaining: 7.19s
78:	learn: 1710.8058930	total: 25.8s	remaining: 6.86s
79:	learn: 1709.7825254	total: 26.1s	remaining: 6.52s
80:	learn: 1708.2844141	total: 26.4s	remaining: 6.19s
81:	learn: 1707.0946594	total: 26.8s	remaining: 5.88s
82:	learn: 1704.8288339	total: 27.1s	remaining: 5.55s
83:	learn: 1703.0598021	total: 27.4s	remaining: 5.22s
84:	learn: 1700.3717096	total: 27.7s	remaining: 4.89s
85:	learn: 1698.0617504	total: 28.1s	remaining: 4.57s
86:	learn: 1695.7498616	total: 28.4s	remaining: 4.24s
87:	learn: 1694.1319657	total: 28.7s	remaining: 3.91s
88:	learn: 1693.3326930	total: 29s	remaining: 3.58s
89:	learn: 1691.4472548	total: 29.4s	remaining: 3.26s
90:	learn: 1689.8062787	total: 29.7s	remaining: 2.94s
91:	learn: 1687.7147595	total: 30s	remaining: 2.61s
92:	learn: 1686.4808478	total: 30.3s	remaining: 2.28s
93:	learn: 1685.8033705	total: 30.6s	remaining: 1.95s
94:	learn: 1684.7222289	total: 31s	remaining: 1.63s
95:	learn: 1681.8468786	total: 31.

25:	learn: 1962.6850194	total: 8.38s	remaining: 23.9s
26:	learn: 1948.0103491	total: 8.68s	remaining: 23.5s
27:	learn: 1934.9086362	total: 8.98s	remaining: 23.1s
28:	learn: 1922.8281234	total: 9.29s	remaining: 22.7s
29:	learn: 1907.9535517	total: 9.59s	remaining: 22.4s
30:	learn: 1896.1874179	total: 9.98s	remaining: 22.2s
31:	learn: 1883.4003720	total: 10.3s	remaining: 21.8s
32:	learn: 1871.1205007	total: 10.6s	remaining: 21.5s
33:	learn: 1861.4158736	total: 10.9s	remaining: 21.1s
34:	learn: 1852.9869282	total: 11.2s	remaining: 20.8s
35:	learn: 1845.2521451	total: 11.5s	remaining: 20.4s
36:	learn: 1838.3901602	total: 11.9s	remaining: 20.2s
37:	learn: 1832.4835585	total: 12.2s	remaining: 19.9s
38:	learn: 1827.0166848	total: 12.5s	remaining: 19.5s
39:	learn: 1822.7779715	total: 12.8s	remaining: 19.2s
40:	learn: 1818.2917735	total: 13.2s	remaining: 18.9s
41:	learn: 1813.1530517	total: 13.5s	remaining: 18.6s
42:	learn: 1808.2554096	total: 13.8s	remaining: 18.3s
43:	learn: 1803.7168099	tota

[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  9.2min finished


0:	learn: 4380.7325613	total: 411ms	remaining: 40.7s
1:	learn: 4087.8244075	total: 810ms	remaining: 39.7s
2:	learn: 3825.8857570	total: 1.22s	remaining: 39.4s
3:	learn: 3598.1767909	total: 1.71s	remaining: 41.1s
4:	learn: 3393.5752622	total: 2.12s	remaining: 40.2s
5:	learn: 3216.2819410	total: 2.61s	remaining: 40.9s
6:	learn: 3063.2489814	total: 3.01s	remaining: 40s
7:	learn: 2932.2079442	total: 3.41s	remaining: 39.2s
8:	learn: 2818.1719280	total: 3.82s	remaining: 38.6s
9:	learn: 2714.5918553	total: 4.31s	remaining: 38.8s
10:	learn: 2608.2146949	total: 4.71s	remaining: 38.1s
11:	learn: 2518.3634317	total: 5.11s	remaining: 37.5s
12:	learn: 2439.9996803	total: 5.61s	remaining: 37.5s
13:	learn: 2372.5194434	total: 6.01s	remaining: 36.9s
14:	learn: 2316.2173467	total: 6.41s	remaining: 36.3s
15:	learn: 2268.5273108	total: 6.81s	remaining: 35.7s
16:	learn: 2219.7849226	total: 7.21s	remaining: 35.2s
17:	learn: 2178.9178396	total: 7.61s	remaining: 34.7s
18:	learn: 2142.5231365	total: 8.1s	rema

* #### LightGBM

In [16]:
lgbm_model = lgb.LGBMRegressor()
                          
lgbm_params = {'depth'        : [6,10],
              'learning_rate' : [0.03, 0.1],
              'n_estimators'  : [60,100] }

random_search = RandomizedSearchCV(lgbm_model, lgbm_params, n_iter=10, n_jobs=-1, cv=3, 
                                   scoring= 'neg_mean_squared_error', verbose = 2, random_state=12345)

random_search.fit(X_train, y_train, categorical_feature= cat_features)

print("\n The best parameters:\n ", random_search.best_params_)

# Predict after fitting RandomizedSearchCV with best parameters
y_pred = random_search.predict(X_valid)
 
score_lgbm = rmse(y_valid, y_pred)

print("\n Score on test data:\n ", score_lgbm)


Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] n_estimators=60, learning_rate=0.03, depth=6 ....................


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..... n_estimators=60, learning_rate=0.03, depth=6, total=   7.4s
[CV] n_estimators=60, learning_rate=0.03, depth=6 ....................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    7.4s remaining:    0.0s


[CV] ..... n_estimators=60, learning_rate=0.03, depth=6, total=   6.5s
[CV] n_estimators=60, learning_rate=0.03, depth=6 ....................
[CV] ..... n_estimators=60, learning_rate=0.03, depth=6, total=   5.0s
[CV] n_estimators=100, learning_rate=0.03, depth=6 ...................
[CV] .... n_estimators=100, learning_rate=0.03, depth=6, total=   8.2s
[CV] n_estimators=100, learning_rate=0.03, depth=6 ...................
[CV] .... n_estimators=100, learning_rate=0.03, depth=6, total=   7.4s
[CV] n_estimators=100, learning_rate=0.03, depth=6 ...................
[CV] .... n_estimators=100, learning_rate=0.03, depth=6, total=   8.1s
[CV] n_estimators=60, learning_rate=0.1, depth=6 .....................
[CV] ...... n_estimators=60, learning_rate=0.1, depth=6, total=   4.6s
[CV] n_estimators=60, learning_rate=0.1, depth=6 .....................
[CV] ...... n_estimators=60, learning_rate=0.1, depth=6, total=   4.8s
[CV] n_estimators=60, learning_rate=0.1, depth=6 .....................
[CV] .

[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  2.5min finished



 The best parameters:
  {'n_estimators': 100, 'learning_rate': 0.1, 'depth': 6}

 Score on test data:
  1664.1104279896954


Let’s find out how long it takes training and prediction for each model.

In [17]:
#Linear regression training
start_time = time.time()

regressor = LinearRegression()  
regressor.fit(features_train, target_train)

elapsed_time = time.time() - start_time
time_LR_t = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
print("Training Time: ", time_LR_t)


Training Time:  00:00:18


In [18]:
#Linear regression prediction
start_time = time.time()

predicted_valid = regressor.predict(features_valid)

elapsed_time = time.time() - start_time
time_LR_p = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
print("Prediction Time: ", time_LR_p)


Prediction Time:  00:00:00


In [19]:
#Random Forest training
start = time.time()

model = RandomForestRegressor(n_estimators = 50, max_depth = 10, random_state=0)
model.fit(features_train, target_train)

elapsed_time = time.time() - start
time_RF_t = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
print("Training Time: ", time_RF_t)


Training Time:  00:02:06


In [20]:
#Random Forest prediction
start = time.time()

y_pred = model.predict(features_valid)

elapsed_time = time.time() - start
time_RF_p = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
print("Prediction Time: ", time_RF_p)

Prediction Time:  00:00:00


In [21]:
#CatBoost training
start = time.time()

model = CatBoostRegressor(n_estimators = 100, learning_rate = 0.1, depth = 10, random_state = 0)

model.fit(X_train, y_train,cat_features=cat_features)

elapsed_time = time.time() - start
time_CatB_t = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
print("Training Time: ", time_CatB_t)

0:	learn: 4380.7325613	total: 465ms	remaining: 46.1s
1:	learn: 4087.8244075	total: 870ms	remaining: 42.6s
2:	learn: 3825.8857570	total: 1.27s	remaining: 41.2s
3:	learn: 3598.1767909	total: 1.77s	remaining: 42.5s
4:	learn: 3393.5752622	total: 2.18s	remaining: 41.4s
5:	learn: 3216.2819410	total: 2.66s	remaining: 41.7s
6:	learn: 3063.2489814	total: 3.07s	remaining: 40.8s
7:	learn: 2932.2079442	total: 3.47s	remaining: 39.9s
8:	learn: 2818.1719280	total: 3.87s	remaining: 39.1s
9:	learn: 2714.5918553	total: 4.36s	remaining: 39.3s
10:	learn: 2608.2146949	total: 4.76s	remaining: 38.6s
11:	learn: 2518.3634317	total: 5.17s	remaining: 37.9s
12:	learn: 2439.9996803	total: 5.66s	remaining: 37.9s
13:	learn: 2372.5194434	total: 6.06s	remaining: 37.2s
14:	learn: 2316.2173467	total: 6.46s	remaining: 36.6s
15:	learn: 2268.5273108	total: 6.87s	remaining: 36s
16:	learn: 2219.7849226	total: 7.35s	remaining: 35.9s
17:	learn: 2178.9178396	total: 7.75s	remaining: 35.3s
18:	learn: 2142.5231365	total: 8.15s	rem

In [22]:
#CatBoost prediction
start = time.time()

y_pred = model.predict(X_valid)

elapsed_time = time.time() - start
time_CatB_p = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
print("Prediction Time: ", time_CatB_p)

Prediction Time:  00:00:00


In [23]:
#LGBM training
start = time.time()

lgbm_model = lgb.LGBMRegressor(n_estimators = 100, learning_rate = 0.1, depth = 6, random_state = 0)

lgbm_model.fit(X_train, y_train, categorical_feature= cat_features)

elapsed_time = time.time() - start
time_LGBM_t = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
print("Training Time: ", time_LGBM_t)

Training Time:  00:00:07


In [24]:
#LGBM prediction
start = time.time()

y_pred = lgbm_model.predict(X_valid)
 
elapsed_time = time.time() - start
time_LGBM_p = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
print("Prediction Time: ", time_LGBM_p)

Prediction Time:  00:00:00


In [25]:
data = {'Model': ['Linear Regression', 'Random Forest', 'CatBoost', 'LightGBM'],
        'RMSE': [rmse_linear_reg, score_random_forest, score_catboost, score_lgbm],
        'Time train': [time_LR_t, time_RF_t, time_CatB_t, time_LGBM_t],
        'Time prediction': [time_LR_p, time_RF_p, time_CatB_p, time_LGBM_p] }

conclusion = pd.DataFrame (data, columns = ['Model', 'RMSE', 'Time train', 'Time prediction'])
conclusion

,Model,RMSE,Time train,Time prediction
0,Linear Regression,2713.527209,00:00:18,00:00:00
1,Random Forest,1946.425322,00:02:06,00:00:00
2,CatBoost,1707.620656,00:00:44,00:00:00
3,LightGBM,1664.110428,00:00:07,00:00:00


We use Linear Regression as a sanity check. All the rest models perfomed better than Linear Regression.

We can see from the table above that LightGBM has the best value of RMSE and also it is the fastest model.

So, the conclusion is to choose LightGBM to predict the market value of the car.